<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/model_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analysis of model results



In [1]:
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [2]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [0]:
from google.colab import files
import io 

Don't need to do the next few cells unless uploading new results from local machine.

In [27]:
# Get manually inputted model results. [11/23/19 3pm]

uploaded = files.upload()
model_results = pd.read_csv(io.BytesIO(uploaded['model_results.csv']))

Saving model_results.csv to model_results (6).csv


In [0]:
# Download the new dataframe to local machine.

model_results.to_csv('model_results.csv', header=True, index=False)
files.download('model_results.csv')

In [28]:
  !gsutil cp model_results\ \(6\).csv gs://fb-congressional-data/models/model_results.csv

Copying file://model_results (6).csv [Content-Type=text/csv]...
/ [1 files][ 21.9 KiB/ 21.9 KiB]                                                
Operation completed over 1 objects/21.9 KiB.                                     


Start here to get results from the bucket.

In [0]:
model_results = pd.read_csv("gs://fb-congressional-data/models/model_results.csv")

In [12]:
model_results.head()

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,best_delta_val_acc,percent_W,percent_D,saved_model_name
0,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,10,5000,2,32,2,0.2,10,50,1,0.5206,0.7662,0.8490,0.4936,0.7781,0.8489,0.8490,1,-0.0001,0.0000,1.559,NaN,test3_model_0_gender_1574450353.2073507.h5
1,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,20,5000,2,32,2,0.2,10,50,2,0.5180,0.7663,0.8501,0.4891,0.7789,0.8472,0.8486,5,-0.0029,-0.0015,1.861,NaN,test3_model_2_gender_1574450353.2073507.h5
2,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,30,5000,2,32,2,0.2,10,50,3,0.5106,0.7699,0.8511,0.4844,0.7813,0.8496,0.8517,3,-0.0015,0.0006,1.994,NaN,LPFriday11_22_model_1_gender_1574459044.4948301.h5
3,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.2,10,50,4,0.5143,0.7677,0.8452,0.4830,0.7820,0.8466,0.8477,4,0.0014,0.0025,2.384,NaN,LPFriday11_22_model_3_gender_1574459044.4948301.h5
4,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,100,5000,2,32,2,0.2,10,50,5,0.5068,0.7718,0.8467,0.4813,0.7829,0.8473,0.8490,3,0.0006,0.0023,2.204,NaN,LPFriday11_22_model_5_gender_1574459044.4948301.h5


In [13]:
model_results.sort_values(['delta_val_acc', 'percent_W'], ascending=[False, False])

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,best_delta_val_acc,percent_W,percent_D,saved_model_name
66,Y,2,0,0,2234119.0,1326906.0,0,0,1517593,524352,50,10,1000,20,5000,1,32,2,0.2,10,50,33,0.6084,0.6656,0.7004,0.5607,0.6973,0.7468,0.7468,10,0.0464,0.0464,NaN,9.7900,emFridayRound1_model_4_party_1574440064.0135477.h5
14,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,100,5000,2,32,2,0.2,10,50,10,0.5966,0.6708,0.6962,0.5561,0.6977,0.7377,0.7466,7,0.0415,0.0504,NaN,14.2997,LPFriday11_22_model_5_party_1574459044.4948301.h5
99,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,3,0.2,10,50,46,0.5820,0.6828,0.7010,0.5408,0.7090,0.7423,0.7423,10,0.0413,0.0413,NaN,14.6170,emSaturdayRound1_model_1_party_1574526502.7369754.h5
64,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,20,5000,1,32,2,0.2,10,50,31,0.6153,0.6573,0.6997,0.5618,0.6956,0.7342,0.7422,9,0.0345,0.0425,NaN,14.9720,emFridayRound1_model_2_party_1574440064.0135477.h5
41,Y,2,0,0,1813217.0,1108843.0,0,0,1698252,594655,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.5846,0.6780,0.7192,0.5363,0.7096,0.7531,0.7531,10,0.0339,0.0339,NaN,12.9760,LPFriday11_22_model_1_party_1574533565.182682.h5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,N,2,0,0,2234119.0,1326906.0,0,0,1517593,524352,50,10,1000,20,5000,1,32,2,0.2,10,50,34,0.6157,0.6595,0.7378,0.5780,0.6856,0.7243,0.7397,2,-0.0135,0.0019,NaN,12.5370,emFridayRound1_model_5_party_1574440064.0135477.h5
27,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,32,2,0.8,10,50,14,0.6613,0.6228,0.6839,0.6110,0.6583,0.6676,0.6839,1,-0.0163,0.0000,NaN,31.8550,LPFriday11_22_model_3_party_1574474133.6134286.h5
90,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,4,8,2,0.2,10,50,37,0.6498,0.6319,0.7204,0.5995,0.6706,0.7017,0.7204,1,-0.0187,0.0000,NaN,9.2200,emFridayRound2_model_2_party_1574448920.8472505.h5
89,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,3,8,3,0.2,10,50,36,0.6372,0.6306,0.6699,0.6177,0.6439,0.5901,0.6782,4,-0.0798,0.0083,NaN,43.4540,emFridayRound2_model_1_party_1574448920.8472505.h5


In [0]:
gender_results = model_results[model_results['saved_model_name'].str.contains("gender")]
party_results = model_results[model_results['saved_model_name'].str.contains("party")]


In [0]:
gender_best_delta = gender_results.sort_values(['delta_val_acc', 'percent_W'], ascending=[False, False]).head(10)
gender_best_final = gender_results.sort_values("final_val_acc", ascending=False).head(10)
gender_best_acc = gender_results.sort_values("best_val_acc", ascending=False).head(10)
gender_best_W = gender_results.sort_values("percent_W", ascending=False).head(10)

party_best_delta = party_results.sort_values(['delta_val_acc', 'percent_D'], ascending=[False, False]).head(10)
party_best_final = party_results.sort_values("final_val_acc", ascending=False).head(10)
party_best_acc = party_results.sort_values("best_val_acc", ascending=False).head(10)
party_best_D = party_results.sort_values("percent_D", ascending=False).head(10)


In [0]:
best_indices = gender_best_delta.model_idx
all_best_indices = best_indices.append([gender_best_final.model_idx,gender_best_acc.model_idx,gender_best_W.model_idx,
                     party_best_delta.model_idx,party_best_final.model_idx,party_best_acc.model_idx,party_best_D.model_idx])
top_models = all_best_indices.value_counts().head(10)

In [63]:
top_models = top_models.reset_index()
top_models

,index,model_idx
0,17,5
1,18,4
2,40,3
3,20,3
4,27,3
5,49,3
6,43,3
7,36,3
8,14,3
9,16,3


In [64]:
model_results.loc[model_results['model_idx'].isin(top_models['index'])]

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
23,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.8,10,50,14,0.5717,0.7407,0.8492,0.5192,0.7666,0.8515,0.8515,10,0.0023,0.0023,0.6293,NaN,LPFriday11_22_model_3_gender_1574474133.6134286.h5
27,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,32,2,0.8,10,50,14,0.6613,0.6228,0.6839,0.6110,0.6583,0.6676,0.6839,1,-0.0163,0.0000,NaN,31.855,LPFriday11_22_model_3_party_1574474133.6134286.h5
29,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.2,10,50,16,0.4919,0.7780,0.8433,0.4696,0.7865,0.8482,0.8482,10,0.0049,0.0049,2.4580,NaN,LPSaturday11_23_model_1_gender_1574532795.1390407.h5
30,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5067,0.7710,0.8483,0.4793,0.7833,0.8510,0.8512,9,0.0027,0.0029,1.1720,NaN,LPSaturday11_23_model_2_gender_1574532795.1390407.h5
31,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.5344,0.7583,0.8448,0.4968,0.7764,0.8516,0.8516,10,0.0068,0.0068,0.7170,NaN,LPSaturday11_23_model_3_gender_1574532795.1390407.h5
33,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.2,10,50,16,0.5723,0.6878,0.7357,0.5307,0.7148,0.7517,0.7540,9,0.0160,0.0183,NaN,14.499,LPSaturday11_23_model_1_party_1574532795.1390407.h5
34,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5901,0.6750,0.7351,0.5482,0.7031,0.7504,0.7505,7,0.0153,0.0154,NaN,7.925,LPSaturday11_23_model_2_party_1574532795.1390407.h5
35,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.6449,0.6367,0.7388,0.5807,0.6817,0.7410,0.7411,5,0.0022,0.0023,NaN,31.855,truncated to 6 epochs; met output size limit; party model
37,Y,2,2168175,765916,NaN,NaN,1948371,344536,0,0,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.4952,0.7785,0.8487,0.4667,0.7902,0.8475,0.8494,9,-0.0012,0.0007,2.7990,NaN,LPFriday11_22_model_1_gender_1574533565.182682.h5
41,Y,2,0,0,1813217.0,1108843.0,0,0,1698252,594655,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.5846,0.6780,0.7192,0.5363,0.7096,0.7531,0.7531,10,0.0339,0.0339,NaN,12.976,LPFriday11_22_model_1_party_1574533565.182682.h5


In [65]:
model_results.loc[model_results['model_idx']==17]


,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
30,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5067,0.771,0.8483,0.4793,0.7833,0.8510,0.8512,9,0.0027,0.0029,1.172,NaN,LPSaturday11_23_model_2_gender_1574532795.1390407.h5
34,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5901,0.675,0.7351,0.5482,0.7031,0.7504,0.7505,7,0.0153,0.0154,NaN,7.925,LPSaturday11_23_model_2_party_1574532795.1390407.h5


In [52]:
model_results.loc[model_results['model_idx']==18]


,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
73,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,3,32,2,0.2,10,50,40,0.5127,0.7688,0.8443,0.4847,0.7817,0.8474,0.8481,2,0.0031,0.0038,2.442,NaN,emFridayRound2_model_5_gender_1574448920.8472505.h5
93,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,3,32,2,0.2,10,50,40,0.6037,0.6665,0.7449,0.5617,0.6946,0.7480,0.7496,7,0.0031,0.0047,NaN,10.541,emFridayRound2_model_5_party_1574448920.8472505.h5


In [34]:
gender_best_final = gender_results.sort_values("final_val_acc", ascending=False).head(10)

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
56,Y,0,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,20,5000,1,32,2,0.2,10,50,29,0.5229,0.7638,0.8515,0.4913,0.7781,0.8516,0.8537,6,0.0001,0.0022,2.42800,NaN,emFridayRound1_model_0_gender_1574440064.0135477.h5
31,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.5344,0.7583,0.8448,0.4968,0.7764,0.8516,0.8516,10,0.0068,0.0068,0.71700,NaN,LPSaturday11_23_model_3_gender_1574532795.1390407.h5
23,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.8,10,50,14,0.5717,0.7407,0.8492,0.5192,0.7666,0.8515,0.8515,10,0.0023,0.0023,0.62930,NaN,LPFriday11_22_model_3_gender_1574474133.6134286.h5
30,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5067,0.7710,0.8483,0.4793,0.7833,0.8510,0.8512,9,0.0027,0.0029,1.17200,NaN,LPSaturday11_23_model_2_gender_1574532795.1390407.h5
48,Y,2,2625121,951327,0.0,0.0,1731288,310657,0,0,50,10,1000,20,5000,1,8,2,0.2,10,50,27,0.5361,0.7579,0.8457,0.5034,0.7742,0.8498,0.8510,5,0.0041,0.0053,3.02030,NaN,test3_model_4_gender_1574374022.0840676.h5
2,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,30,5000,2,32,2,0.2,10,50,3,0.5106,0.7699,0.8511,0.4844,0.7813,0.8496,0.8517,3,-0.0015,0.0006,1.99400,NaN,LPFriday11_22_model_1_gender_1574459044.4948301.h5
21,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.2,10,50,12,0.5097,0.7711,0.8484,0.4825,0.7821,0.8493,0.8493,10,0.0009,0.0009,1.96980,NaN,LPFriday11_22_model_1_gender_1574474133.6134286.h5
45,N,0,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,20,5000,1,8,2,0.2,10,50,24,0.5439,0.7529,0.8463,0.5224,0.7641,0.8492,0.8492,10,0.0029,0.0029,1.52614,NaN,test3_model_1_gender_1574374022.0840676.h5
69,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,3,8,3,0.2,10,50,36,0.5296,0.7609,0.8484,0.5039,0.7737,0.8490,0.8509,7,0.0006,0.0025,2.48200,NaN,emFridayRound2_model_1_gender_1574448920.8472505.h5
0,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,10,5000,2,32,2,0.2,10,50,1,0.5206,0.7662,0.8490,0.4936,0.7781,0.8489,0.8490,1,-0.0001,0.0000,1.55900,NaN,test3_model_0_gender_1574450353.2073507.h5


In [35]:
gender_best_acc = gender_results.sort_values("best_val_acc", ascending=False).head(10)

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
56,Y,0,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,20,5000,1,32,2,0.2,10,50,29,0.5229,0.7638,0.8515,0.4913,0.7781,0.8516,0.8537,6,0.0001,0.0022,2.4280,NaN,emFridayRound1_model_0_gender_1574440064.0135477.h5
2,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,30,5000,2,32,2,0.2,10,50,3,0.5106,0.7699,0.8511,0.4844,0.7813,0.8496,0.8517,3,-0.0015,0.0006,1.9940,NaN,LPFriday11_22_model_1_gender_1574459044.4948301.h5
31,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.8,10,50,18,0.5344,0.7583,0.8448,0.4968,0.7764,0.8516,0.8516,10,0.0068,0.0068,0.7170,NaN,LPSaturday11_23_model_3_gender_1574532795.1390407.h5
23,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.8,10,50,14,0.5717,0.7407,0.8492,0.5192,0.7666,0.8515,0.8515,10,0.0023,0.0023,0.6293,NaN,LPFriday11_22_model_3_gender_1574474133.6134286.h5
30,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.5,10,50,17,0.5067,0.7710,0.8483,0.4793,0.7833,0.8510,0.8512,9,0.0027,0.0029,1.1720,NaN,LPSaturday11_23_model_2_gender_1574532795.1390407.h5
48,Y,2,2625121,951327,0.0,0.0,1731288,310657,0,0,50,10,1000,20,5000,1,8,2,0.2,10,50,27,0.5361,0.7579,0.8457,0.5034,0.7742,0.8498,0.8510,5,0.0041,0.0053,3.0203,NaN,test3_model_4_gender_1574374022.0840676.h5
69,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,3,8,3,0.2,10,50,36,0.5296,0.7609,0.8484,0.5039,0.7737,0.8490,0.8509,7,0.0006,0.0025,2.4820,NaN,emFridayRound2_model_1_gender_1574448920.8472505.h5
83,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,1,128,3,0.5,10,50,50,0.5085,0.7707,0.8484,0.4823,0.7823,0.8489,0.8509,3,0.0005,0.0025,2.3310,NaN,emSaturdayRound1_model_5_gender_1574526502.7369754.h5
72,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,2,32,3,0.2,10,50,39,0.5085,0.7716,0.8506,0.4836,0.7821,0.8478,0.8506,1,-0.0028,0.0000,2.0400,NaN,emFridayRound2_model_4_gender_1574448920.8472505.h5
28,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.0,10,50,15,0.4843,0.7809,0.8502,0.4597,0.7899,0.8408,0.8502,1,-0.0094,0.0000,3.9940,NaN,LPSaturday11_23_model_0_gender_1574532795.1390407.h5


In [37]:
gender_best_W = gender_results.sort_values("percent_W", ascending=False).head(10)

,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,delta_best_val_acc,percent_W,percent_D,saved_model_name
85,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,2,256,4,0.2,10,50,52,0.5026,0.7758,0.8357,0.4640,0.7892,0.8338,0.8422,3,-0.0019,0.0065,5.1860,NaN,emSaturdayRound2_model_1_gender_1574533378.3978605.h5
70,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,4,8,2,0.2,10,50,37,0.5626,0.7326,0.8471,0.5300,0.7538,0.8455,0.8471,3,-0.0016,0.0000,4.7160,NaN,emFridayRound2_model_2_gender_1574448920.8472505.h5
28,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,128,2,0.0,10,50,15,0.4843,0.7809,0.8502,0.4597,0.7899,0.8408,0.8502,1,-0.0094,0.0000,3.9940,NaN,LPSaturday11_23_model_0_gender_1574532795.1390407.h5
81,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,1,128,5,0.2,10,50,48,0.5032,0.7737,0.8487,0.4748,0.7851,0.8409,0.8487,1,-0.0078,0.0000,3.9490,NaN,emSaturdayRound1_model_3_gender_1574526502.7369754.h5
87,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,1,256,3,0.2,10,50,54,0.5005,0.7746,0.8478,0.4727,0.7858,0.8446,0.8478,1,-0.0032,0.0000,3.6880,NaN,emSaturdayRound2_model_3_gender_1574533378.3978605.h5
46,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,20,5000,1,8,2,0.2,10,50,25,0.5349,0.7563,0.8439,0.5045,0.7739,0.8447,0.8488,6,0.0008,0.0049,3.6135,NaN,test3_model_2_gender_1574374022.0840676.h5
58,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,20,5000,1,32,2,0.2,10,50,31,0.5143,0.7686,0.8433,0.4870,0.7810,0.8444,0.8490,2,0.0011,0.0057,3.6110,NaN,emFridayRound1_model_2_gender_1574440064.0135477.h5
38,Y,2,2900753,1061531,0.0,0.0,1948371,344536,0,0,50,10,1000,50,5000,2,32,2,0.2,10,50,21,0.4927,0.7774,0.8466,0.4698,0.7866,0.8439,0.8481,2,-0.0027,0.0015,3.5550,NaN,LPFriday11_22_model_2_gender_1574533565.182682.h5
60,Y,2,2625121,951327,0.0,0.0,1731288,310657,0,0,50,10,1000,20,5000,1,32,2,0.2,10,50,33,0.5127,0.7701,0.8467,0.4830,0.7829,0.8465,0.8485,3,-0.0002,0.0018,3.5380,NaN,emFridayRound1_model_4_gender_1574440064.0135477.h5
80,Y,1,2751507,1004243,0.0,0.0,1821965,328827,0,0,50,10,1000,50,20,1,128,4,0.2,10,50,47,0.5005,0.7743,0.8490,0.4754,0.7849,0.8432,0.8490,1,-0.0058,0.0000,3.4220,NaN,emSaturdayRound1_model_2_gender_1574526502.7369754.h5


In [17]:
party_results.sort_values(['delta_val_acc', 'percent_D'], ascending=[False, False]).head(10)


,beg_end_resp,drop_short,train_M,train_W,train_R,train_D,dev_M,dev_W,dev_R,dev_D,max_resps,epochs,batch_size,max_seq,freq_wds,n_layers,layer_size,filter_size,dropout,dense_size,embedding_dim,model_idx,init_loss,init_acc,init_val_acc,final_loss,final_acc,final_val_acc,best_val_acc,best_epoch,delta_val_acc,best_delta_val_acc,percent_W,percent_D,saved_model_name
66,Y,2,0,0,2234119.0,1326906.0,0,0,1517593,524352,50,10,1000,20,5000,1,32,2,0.2,10,50,33,0.6084,0.6656,0.7004,0.5607,0.6973,0.7468,0.7468,10,0.0464,0.0464,NaN,9.7900,emFridayRound1_model_4_party_1574440064.0135477.h5
14,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,100,5000,2,32,2,0.2,10,50,10,0.5966,0.6708,0.6962,0.5561,0.6977,0.7377,0.7466,7,0.0415,0.0504,NaN,14.2997,LPFriday11_22_model_5_party_1574459044.4948301.h5
99,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,3,0.2,10,50,46,0.5820,0.6828,0.7010,0.5408,0.7090,0.7423,0.7423,10,0.0413,0.0413,NaN,14.6170,emSaturdayRound1_model_1_party_1574526502.7369754.h5
64,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,20,5000,1,32,2,0.2,10,50,31,0.6153,0.6573,0.6997,0.5618,0.6956,0.7342,0.7422,9,0.0345,0.0425,NaN,14.9720,emFridayRound1_model_2_party_1574440064.0135477.h5
41,Y,2,0,0,1813217.0,1108843.0,0,0,1698252,594655,25,10,1000,50,5000,2,32,2,0.2,10,50,20,0.5846,0.6780,0.7192,0.5363,0.7096,0.7531,0.7531,10,0.0339,0.0339,NaN,12.9760,LPFriday11_22_model_1_party_1574533565.182682.h5
32,Y,2,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,50,5000,2,128,2,0.0,10,50,15,0.5535,0.7008,0.7103,0.5125,0.7269,0.7416,0.7416,10,0.0313,0.0313,NaN,15.1070,LPSaturday11_23_model_0_party_1574532795.1390407.h5
98,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,2,0.2,10,50,45,0.5841,0.6810,0.7163,0.5418,0.7085,0.7443,0.7443,10,0.0280,0.0280,NaN,13.4500,emSaturdayRound1_model_0_party_1574526502.7369754.h5
100,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,4,0.2,10,50,47,0.5853,0.6800,0.7108,0.5419,0.7085,0.7376,0.7430,9,0.0268,0.0322,NaN,17.2630,emSaturdayRound1_model_2_party_1574526502.7369754.h5
102,Y,1,0,0,2336689.0,1402732.0,0,0,1593280,557512,50,10,1000,50,20,1,128,2,0.5,10,50,49,0.5997,0.6701,0.7215,0.5539,0.7008,0.7479,0.7515,9,0.0264,0.0300,NaN,13.9970,emSaturdayRound1_model_4_party_1574526502.7369754.h5
63,N,0,0,0,2459013.0,1486355.0,0,0,1698252,594655,50,10,1000,20,5000,1,32,2,0.2,10,50,30,0.6143,0.6570,0.7131,0.5835,0.6788,0.7392,0.7402,8,0.0261,0.0271,NaN,12.8440,emFridayRound1_model_1_party_1574440064.0135477.h5


In [14]:
len(party_results)

53

In [15]:
len(gender_results)

54